In [ ]:
# load an existing experiment configuration file
CONFIG_FILE = (
    "../../../experiments/interpretation/dutchf3_patch/configs/seresnet_unet.yaml"
)
# number of images to score
N_EVALUATE = 20
# demo flag - by default notebook runs in demo mode and only fine-tunes the pre-trained model. Set to False for full re-training.
DEMO = True
# options are test1 or test2 - picks which Dutch F3 test set split to use
TEST_SPLIT = "test1"

import os
assert os.path.isfile(CONFIG_FILE), "Experiment config file CONFIG_FILE not found on disk"
assert isinstance(N_EVALUATE, int) and N_EVALUATE>0, "Number of images to score has to be a positive integer"
assert isinstance(DEMO, bool), "demo mode should be a boolean"
assert TEST_SPLIT == "test1" or TEST_SPLIT == "test2"

In [ ]:
#library imports
import numpy as np
import torch
import logging
import logging.config
from os import path

import random
import matplotlib.pyplot as plt

plt.rcParams.update({"font.size": 16})

import yacs.config

import cv2
from albumentations import Compose, HorizontalFlip, Normalize, PadIfNeeded, Resize
from ignite.contrib.handlers import CosineAnnealingScheduler
from ignite.handlers import ModelCheckpoint
from ignite.engine import Events
from ignite.metrics import Loss
from ignite.utils import convert_tensor
from toolz import compose
from torch.utils import data

from cv_lib.utils import load_log_configuration
from cv_lib.event_handlers import SnapshotHandler, logging_handlers
from cv_lib.event_handlers.logging_handlers import Evaluator
from cv_lib.event_handlers import tensorboard_handlers
from cv_lib.event_handlers.tensorboard_handlers import create_summary_writer
from cv_lib.segmentation import models
from cv_lib.segmentation.dutchf3.engine import (
    create_supervised_evaluator,
    create_supervised_trainer,
)

from cv_lib.segmentation.metrics import (
    pixelwise_accuracy,
    class_accuracy,
    mean_class_accuracy,
    class_iou,
    mean_iou,
)

from cv_lib.segmentation.dutchf3.utils import (
    current_datetime,    
    git_branch,
    git_hash,
    np_to_tb,
)

from cv_lib.utils import generate_path

from deepseismic_interpretation.dutchf3.data import (
    get_patch_loader,    
    get_test_loader,
)

from itkwidgets import view

from utilities import (
    plot_aline,
    patch_label_2d,
    compose_processing_pipeline,
    output_processing_pipeline,
    write_section_file,
    runningScore,
    validate_config_paths,
    download_pretrained_model,
)


In [ ]:
#experiment configuration file
with open(CONFIG_FILE, "rt") as f_read:
    config = yacs.config.load_cfg(f_read)

print(
    f"Configuration loaded. Please check that the DATASET.ROOT:{config.DATASET.ROOT} points to your data location."
)
print(
    f"To modify any of the options, please edit the configuration file {CONFIG_FILE} and reload. \n"
)

In [ ]:
# The number of datapoints you want to run in training or validation per batch
# Setting to None will run whole dataset
# useful for integration tests with a setting of something like 3
# Use only if you want to check things are running and don't want to run
# through whole dataset
# The number of epochs to run in training
max_epochs = config.TRAIN.END_EPOCH
max_snapshots = config.TRAIN.SNAPSHOTS
papermill = False
dataset_root = config.DATASET.ROOT
model_pretrained = config.MODEL.PRETRAINED if "PRETRAINED" in config.MODEL.keys() else None

In [ ]:
# read back the parameters from papermill to config if papermill was used to run this notebook
if papermill:
    # reduce number of images scored for testing
    N_EVALUATE=2

opts = [
    "DATASET.ROOT",
    dataset_root,
    "TRAIN.END_EPOCH",
    max_epochs,
    "TRAIN.SNAPSHOTS",
    max_snapshots,
]
if "PRETRAINED" in config.MODEL.keys():
    opts += ["MODEL.PRETRAINED", model_pretrained]

config.merge_from_list(opts)

# download pre-trained model if possible
config = download_pretrained_model(config)

# update model pretrained (in case it was changed when the pretrained model was downloaded)
model_pretrained = config.MODEL.PRETRAINED

In [ ]:
if DEMO:
    opts = [
        "TRAIN.END_EPOCH",
        1,
        "TRAIN.SNAPSHOTS",
        1,
        "TRAIN.MAX_LR",
        10 ** -9,
        "TRAIN.MIN_LR",
        10 ** -9,
    ]
    config.merge_from_list(opts)

In [ ]:
# Fix random seeds, and set CUDNN benchmark mode:
torch.backends.cudnn.benchmark = config.CUDNN.BENCHMARK

# Fix random seeds:
torch.manual_seed(config.SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(config.SEED)
np.random.seed(seed=config.SEED)

In [ ]:
print(config)
validate_config_paths(config)

In [ ]:
# Load training data and labels
train_seismic = np.load(path.join(config.DATASET.ROOT, "train/train_seismic.npy"))
train_labels = np.load(path.join(config.DATASET.ROOT, "train/train_labels.npy"))

print(f"Number of inline slices: {train_seismic.shape[0]}")
print(f"Number of crossline slices: {train_seismic.shape[1]}")
print(f"Depth dimension : {train_seismic.shape[2]}")

In [ ]:
view(train_labels, slicing_planes=True)

In [ ]:
#plot a crossline slice
idx = 100
x_in = train_seismic[idx, :, :].swapaxes(0, 1)
x_inl = train_labels[idx, :, :].swapaxes(0, 1)

plot_aline(x_in, x_inl, xlabel="crossline (relative)")

In [ ]:
#plot an inline slice
x_cr = train_seismic[:, idx, :].swapaxes(0, 1)
x_crl = train_labels[:, idx, :].swapaxes(0, 1)

plot_aline(x_cr, x_crl, xlabel="inline (relative)")

In [ ]:
#load the data and apply energy attribute 
scheduler_step = config.TRAIN.END_EPOCH // config.TRAIN.SNAPSHOTS

TrainPatchLoader = get_patch_loader(config)

train_set = TrainPatchLoader(
    config,
    split="train",
    is_transform=True,
    augmentations=train_aug,
)
n_classes = train_set.n_classes
logger.info(train_set, energy)
val_set = TrainPatchLoader(
    config,
    split="val",
    is_transform=True,
    augmentations=val_aug,
)

if papermill:
    train_set = data.Subset(train_set, range(3))
    val_set = data.Subset(val_set, range(3))
elif DEMO:
    val_set = data.Subset(val_set, range(config.VALIDATION.BATCH_SIZE_PER_GPU))

logger.info(val_set, energy)

train_loader = data.DataLoader(
    train_set, energy
    batch_size=config.TRAIN.BATCH_SIZE_PER_GPU,
    num_workers=config.WORKERS,
    shuffle=True,
)
val_loader = data.DataLoader(
    val_set, energy
    batch_size=config.VALIDATION.BATCH_SIZE_PER_GPU,
    num_workers=config.WORKERS,
)